In [5]:
%pip install --quiet pydantic
from pathlib import Path
import json

product_path = Path("../initial-training-sets/datasets/dataset_products.json")
product_images_path = Path("../initial-training-sets/datasets/images/")
product_data = json.loads(product_path.read_text())

from datetime import date
from uuid import UUID
from pydantic import BaseModel
from typing import List

class RawReview(BaseModel):
    id: UUID
    review_ref: str
    product_id: UUID
    review_content: str
    review_title: str
    date_written: date
    product_asin: str
    helpful_count: int
    rating_given: int
    review_page_url: str

class RAWProduct(BaseModel):
    id: UUID
    name: str
    description: str
    product_asin: str
    overall_ratings: float
    total_customers_that_rated: int
    price: float
    currency: str
    category: str
    sub_category: str
    product_page_url: str
    image_url: str
    reviews: List[RawReview]
    

from langchain_core.documents import Document

class ProductCombinedInformation:
    
    product: RAWProduct
    
    def __init__(self, product: RAWProduct) -> None:
        self.product = product
        
    def get_document(self):
        return Document(page_content=self.info(), id=str(self.product.id), metadata={
            "product_id": self.product.id,
            "product_asin": self.product.product_asin,
            "image_url": self.product.image_url,
            "name": self.product.name
        })
        
    def info(self):
        return (
            f"Product Name: {str(self.product.name).strip()} \n"
            f"Product Description: {str(self.product.description).strip()} \n"
            f"Product Asin: {self.product.product_asin} \n"
            f"Overall Ratings {self.product.overall_ratings} \n"
            f"Total Customers that rated: {self.product.total_customers_that_rated} \n"
            f"Pric: {self.product.currency}{self.product.price} \n"   
        )
    


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def summarise_product_info(product_info: str, model_name: str):
    
    if model_name not in [ "llava", "gpt-4o", "llama3", "llama3.1", "phi3", "mistral" ]:
        raise ValueError("Please provide either llava or gpt-4o")
    
    prompt_template = ("You are an assistant tasked with summarizing tables and "
                    "text for retrieval. These summaries will be embedded and used " 
                    "to retrieve the raw text. Give a concise summary of the product. "
                    "Ensure you include important details "
                    "that is well optimized for retrieval.\n"
                    f"Product: {product_info}\n"
                    "Your response should start with the product name."
                    )
    
    prompt = ChatPromptTemplate.from_template(prompt_template)
    
    if model_name == "gpt-4o":
        model = ChatOpenAI(model=model_name, max_tokens=1024, temperature=0)
    else:
        model = ChatOllama(model=model_name, temperature=0)
    
        
    # { "product_info": RunnablePassthrough() } is same as lambda x: x
    # basically do not modify what the user entered, pass into the key product_info
    summarise_chain = { "product_info": RunnablePassthrough() } | prompt | model | StrOutputParser()
    summary = summarise_chain.invoke(product_info)
    return summary
    

In [19]:
for product in product_data['MEN_FASHION_WITH_REVIEWS']['products'][0:1]:
    combined_product_info = ProductCombinedInformation(product=RAWProduct.model_validate(product))
    
    for model in [ "llava", "gpt-4o", "llama3", "llama3.1", "phi3", "mistral" ]:
        print("\n\n", model)
        print(summarise_product_info(combined_product_info.info(), model ))
    
    
    



 llava
 Product Name: Safety Trainers Men Women Steel Toe Cap Trainers Safety Shoes Lightweight Work Shoes Safety Boots Industrial Protective Shoes

Product Description: Breathable upper, anti-slip pattern design, strong grip not afraid of slippery, night reflective function. Perfect for construction site, exploitation site, forging workshop, manufacture, logistics, warehouse, factory, gardening etc. in the tough working environment. 


 gpt-4o
**Product Name:** Safety Trainers Men Women Steel Toe Cap Trainers Safety Shoes Lightweight Work Shoes Safety Boots Industrial Protective Shoes

**Summary:** These unisex safety trainers are designed for industrial and outdoor use, offering a blend of protection, comfort, and style. Key features include a breathable mesh upper for heat dispersion, a widened steel toe cap that absorbs forces up to 200J and compression of 15000N, and a Kevlar midsole that resists punctures up to 1200N. The trainers are lightweight (780g per pair) with TPR soles 